In [125]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [126]:
import seaborn as sns

LOAN STATUS

In [158]:
df = pd.read_csv("loan_status_train.csv")

In [159]:
df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [160]:
df.isna().sum()

Loan_ID               0
Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

In [161]:
df.isna().sum() / df.shape[0]

Loan_ID              0.000000
Gender               0.021173
Married              0.004886
Dependents           0.024430
Education            0.000000
Self_Employed        0.052117
ApplicantIncome      0.000000
CoapplicantIncome    0.000000
LoanAmount           0.035831
Loan_Amount_Term     0.022801
Credit_History       0.081433
Property_Area        0.000000
Loan_Status          0.000000
dtype: float64

In [162]:
df["Gender"].value_counts()

Male      489
Female    112
Name: Gender, dtype: int64

In [163]:
(df["Gender"].value_counts()/df["Gender"].count() * df.groupby(["Gender"]).mean().values[:,0][::-1]).mean()

2648.409317803661

In [164]:
df["ApplicantIncome"].astype(int)

0      5849
1      4583
2      3000
3      2583
4      6000
       ... 
609    2900
610    4106
611    8072
612    7583
613    4583
Name: ApplicantIncome, Length: 614, dtype: int32

In [165]:
df.loc[(df["ApplicantIncome"]<=2648) & (df["Gender"].isna()), "Gender"] = "Female"
df.loc[(df["ApplicantIncome"]>2648) & (df["Gender"].isna()), "Gender"] = "Male"

In [166]:
df.isna().sum()

Loan_ID               0
Gender                0
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

In [167]:
df.groupby(["Dependents"])["ApplicantIncome"].mean()

Dependents
0     4917.423188
1     5962.274510
2     4926.782178
3+    8581.215686
Name: ApplicantIncome, dtype: float64

In [168]:
df["Dependents"].value_counts().idxmax()

'0'

In [169]:
df["Dependents"].fillna(0, inplace=True)

In [170]:
df.isna().sum()

Loan_ID               0
Gender                0
Married               3
Dependents            0
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

In [171]:
df.groupby(["Self_Employed"]).mean()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History
Self_Employed,,,,,
No,5049.748000,1580.989840,141.748963,343.263804,0.835853
Yes,7380.817073,1501.341463,172.000000,336.300000,0.840000


In [172]:
self_employed_thresh = (df["Self_Employed"].value_counts()/df["Self_Employed"].count() * df.groupby(["Self_Employed"])["ApplicantIncome"].mean().values).sum()

In [173]:
self_employed_thresh

5378.180412371133

In [174]:
df.loc[(df["Self_Employed"].isna()) & (df["ApplicantIncome"]>self_employed_thresh), "Self_Employed"] = "Yes"
df.loc[(df["Self_Employed"].isna()) & (df["ApplicantIncome"]<=self_employed_thresh), "Self_Employed"] = "No"

In [176]:
df["Married"].fillna("No", inplace=True)

In [177]:
df.isna().sum()

Loan_ID               0
Gender                0
Married               0
Dependents            0
Education             0
Self_Employed         0
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

In [178]:
df["LoanAmount"].fillna(df["LoanAmount"].mean(), inplace=True)
df["Loan_Amount_Term"].fillna(df["Loan_Amount_Term"].mean(), inplace=True)
df["Credit_History"].fillna(df["Credit_History"].mean(), inplace=True)

In [216]:
cat_names = ["Gender", "Married", "Education", "Self_Employed", "Property_Area", "Loan_Status"] 
dep_var = ["Loan_Status"]
features = list(set(df)-{"Loan_ID", dep_var[0]})

In [180]:
bkp_df = df.copy()

In [200]:
for cat in cat_names:
    print(f"\"{cat}\":{set(df[cat].unique())},")

"Gender":{'Female', 'Male'},
"Married":{'No', 'Yes'},
"Education":{'Not Graduate', 'Graduate'},
"Self_Employed":{'No', 'Yes'},
"Property_Area":{'Semiurban', 'Urban', 'Rural'},
"Loan_Status":{'Y', 'N'},


In [201]:
replace_map = {
    "Gender":{'Female':0, 'Male':1},
    "Married":{'No':0, 'Yes':1},
    "Education":{'Not Graduate':0, 'Graduate':1},
    "Self_Employed":{'No':0, 'Yes':1},
    "Property_Area":{'Semiurban':0, 'Urban':1, 'Rural':2},
    "Loan_Status":{'Y':1, 'N':0},
}

In [204]:
df.replace(replace_map, inplace=True)

In [222]:
df.loc[(df["Dependents"]=="3+"), "Dependents"]=3

In [213]:
list(set(df)-{"Loan_ID", "Loan_Status"})

['LoanAmount',
 'Education',
 'Married',
 'Property_Area',
 'Credit_History',
 'Dependents',
 'Gender',
 'CoapplicantIncome',
 'Self_Employed',
 'Loan_Amount_Term',
 'ApplicantIncome']

In [232]:
new_df = df.drop("Loan_ID", 1)

In [233]:
from sklearn import preprocessing

In [234]:
x = new_df.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
scaled_df = pd.DataFrame(x_scaled)

C:\Users\Suraj\Miniconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [236]:
scaled_df.columns = new_df.columns

In [303]:
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPClassifier
from sklearn import decomposition
from sklearn.metrics import accuracy_score

In [334]:
x_train, x_test, y_train, y_test = train_test_split(scaled_df[features], scaled_df[dep_var], test_size=0.2, random_state=42)

In [335]:
X_train, Y_train = x_train.values, y_train.values.ravel()
X_test, Y_test = x_test.values, y_test.values.ravel()

In [350]:
random_forest = RandomForestClassifier(max_depth=7,n_estimators=100)
random_forest.fit(X_train, Y_train)

Y_prediction = random_forest.predict(X_test)

print("Eval score: ",accuracy_score(Y_test, Y_prediction))

Eval score:  0.7804878048780488


In [349]:
clf = MLPClassifier(hidden_layer_sizes=(8,8,8), 
                    activation='relu', 
                    solver='adam', max_iter=500
                   )
clf.fit(X_train, Y_train)

Y_prediction = clf.predict(X_test)

print("Eval score: ",accuracy_score(Y_test, Y_prediction))

Eval score:  0.7967479674796748


C:\Users\Suraj\Miniconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [351]:
clf = LogisticRegression()
clf.fit(X_train, Y_train)

Y_prediction = clf.predict(X_test)

print("Eval score: ",accuracy_score(Y_test, Y_prediction))

Eval score:  0.7886178861788617


C:\Users\Suraj\Miniconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [339]:
from catboost import CatBoostClassifier

In [371]:
model = CatBoostClassifier(eval_metric="Accuracy")
model.fit(X_train, Y_train)
preds = model.predict(X_train)

print("Train Accuracy: ",accuracy_score(Y_train, preds))